In [1]:
import pandas as pd
import numpy as np

In [2]:
product_info = pd.read_csv('../input/products.csv')
num_of_depts = len(product_info['department_id'].value_counts())
temp = product_info[['product_id','department_id']]

In [3]:
product_info.head()

,product_id,product_name,aisle_id,department_id
0,9327,Garlic Powder,104,13
1,17461,Air Chilled Organic Boneless Skinless Chicken ...,35,12
2,17668,Unsweetened Chocolate Almond Breeze Almond Milk,91,16
3,28985,Michigan Organic Kale,83,4
4,32665,Organic Ezekiel 49 Bread Cinnamon Raisin,112,3


In [4]:
order_info = pd.read_csv('../input/order_products.csv')

In [5]:
product_info_with_dept = order_info.merge(temp, on='product_id', how='left')

In [6]:
temp = product_info_with_dept.department_id.value_counts(sort=False)
dept_id = temp.index # get the department index
num_of_orders = temp.values # get the values corresponding to the index
ash = np.argsort(dept_id)
otpt = np.concatenate((dept_id[ash], num_of_orders[ash])).reshape(2, len(dept_id)).T # make a data table
report = pd.DataFrame(otpt, pd.Series(otpt[:,0]), pd.Series(['department_id', 'number_of_orders']))

In [7]:
temp = product_info_with_dept[product_info_with_dept.reordered == 0].department_id.value_counts(sort=False)
dept_id = temp.index
num_of_first_orders = temp.values
otpt = np.concatenate((dept_id, num_of_first_orders)).reshape(2, len(dept_id)).T
temp_report = pd.DataFrame(otpt, pd.Series(dept_id), pd.Series(['department_id', 'number_of_first_orders']))
temp_report

,department_id,number_of_first_orders
13,13,1
3,3,1


In [8]:
report = report.merge(temp_report, on='department_id', how='left')
report = report.fillna(0)
report

,department_id,number_of_orders,number_of_first_orders
0,3,2,1.0
1,4,2,0.0
2,12,1,0.0
3,13,2,1.0
4,16,2,0.0


In [9]:
pct = np.concatenate((report.department_id, report['number_of_first_orders']/report['number_of_orders'])).reshape(2,len(report.department_id)).T
temp_report = pd.DataFrame(pct, pd.Series(pct[:,0]), pd.Series(['department_id', 'percentage']))
temp_report.department_id = temp_report.department_id.astype(int)

In [10]:
report = report.merge(temp_report, on='department_id', how='left')
report

,department_id,number_of_orders,number_of_first_orders,percentage
0,3,2,1.0,0.5
1,4,2,0.0,0.0
2,12,1,0.0,0.0
3,13,2,1.0,0.5
4,16,2,0.0,0.0


In [11]:
report.percentage = report.percentage.round(decimals=2)
report.number_of_first_orders = report.number_of_first_orders.astype(np.int64)
report.number_of_orders = report.number_of_orders.astype(np.int64)

In [13]:
pd.options.display.float_format = '{:,.2f}'.format
report

,department_id,number_of_orders,number_of_first_orders,percentage
0,3,2,1,0.50
1,4,2,0,0.00
2,12,1,0,0.00
3,13,2,1,0.50
4,16,2,0,0.00


In [14]:
report = report.set_index('department_id')
report

,number_of_orders,number_of_first_orders,percentage
department_id,,,
3,2,1,0.50
4,2,0,0.00
12,1,0,0.00
13,2,1,0.50
16,2,0,0.00


In [15]:
report.to_csv('../output/report.csv')